<a href="https://colab.research.google.com/github/amruthab91/spark_basics/blob/main/Ensembling_tech_using_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ensemble methods
1) Using one hot encoding

2) LR

3) Decision Tree Classifier

4) AUC - ROC curve

5) Bagging using Random Forest

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=f2d1bd9da0e52e43e5cf204ecb547ffda5ea2c4596dfa7dfd5e523963e63303b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import os
# starts the spark only three steps


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
dff = spark.read.csv('/content/bank.csv',inferSchema=True, header =True)

In [6]:
print(dff.count())

11162


In [7]:
dff.select("deposit").distinct().show()
dff.printSchema()

+-------+
|deposit|
+-------+
|     no|
|    yes|
+-------+

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



check classes are balanced or Not

In [8]:
print("yes",dff.select("deposit").where("deposit=='yes'").count())
print("No",dff.select("deposit").where("deposit=='no'").count())

yes 5289
No 5873


checking the distinct values in all string columns such as:

1)Jobs

2)Education

3)martial status

etc of the dataframe using functional programming

In [10]:
# checking the distinct values in string columns of the dataframe using functional programming
[print("column name",dff[t[0]].name,dff.select(t[0]).distinct().show()) for t in dff.dtypes if t[1]=='string' ]

+-------------+
|          job|
+-------------+
|   management|
|      retired|
|      unknown|
|self-employed|
|      student|
|  blue-collar|
| entrepreneur|
|       admin.|
|   technician|
|     services|
|    housemaid|
|   unemployed|
+-------------+

column name <bound method alias of Column<'job'>> None
+--------+
| marital|
+--------+
|divorced|
| married|
|  single|
+--------+

column name <bound method alias of Column<'marital'>> None
+---------+
|education|
+---------+
|  unknown|
| tertiary|
|secondary|
|  primary|
+---------+

column name <bound method alias of Column<'education'>> None
+-------+
|default|
+-------+
|     no|
|    yes|
+-------+

column name <bound method alias of Column<'default'>> None
+-------+
|housing|
+-------+
|     no|
|    yes|
+-------+

column name <bound method alias of Column<'housing'>> None
+----+
|loan|
+----+
|  no|
| yes|
+----+

column name <bound method alias of Column<'loan'>> None
+---------+
|  contact|
+---------+
|  unknown|
| cell

[None, None, None, None, None, None, None, None, None, None]

Displaying only integer columns

In [11]:
# taking only integer features
numeric_features = [t[0] for t in dff.dtypes if t[1] == 'int']
dff.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,11162,41.231947679627304,11.913369192215518,18,95
balance,11162,1528.5385235620856,3225.413325946149,-6847,81204
day,11162,15.658036194230425,8.420739541006462,1,31
duration,11162,371.99381831213043,347.12838571630687,2,3881
campaign,11162,2.508421429851281,2.7220771816614824,1,63
pdays,11162,51.33040673714388,108.75828197197717,-1,854
previous,11162,0.8325568894463358,2.292007218670508,0,58


In [12]:
#find out is there any null value in any column
print("total records:",dff.count())
[print(col,"Total null values",dff.where(dff[col].isNull()).count()) for col in dff.columns]

total records: 11162
age Total null values 0
job Total null values 0
marital Total null values 0
education Total null values 0
default Total null values 0
balance Total null values 0
housing Total null values 0
loan Total null values 0
contact Total null values 0
day Total null values 0
month Total null values 0
duration Total null values 0
campaign Total null values 0
pdays Total null values 0
previous Total null values 0
poutcome Total null values 0
deposit Total null values 0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Selecting Feature for Feature Vector:

Input variables: age, job, marital, education, default, balance, housing, loan, contact, day, month, duration, campaign, pdays, previous, poutcome. Out PutVariable:variable: deposit , we will keep all of numeric variable for the model. However, day and month columns are not really useful, we will remove these two columns.

In [13]:
# Non Numeric features
df = dff.select('job', 'marital', 'education', 'default',  'housing', 'loan', 'contact', 'duration',  'poutcome', 'deposit')
cols = df.columns
df.printSchema()

root
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



Taking both Numeric and String cols

In [14]:
df = dff.select('age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit')
cols = df.columns
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



Change Categorial Variable using one hot encoding

Step 1: apply string index

Step 2: using String indexing to apply one hot Encoding

Step 3: Create Vector Assembler

In [15]:
# Now we will apply string indexing to categorial variables using pipe line
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    print(stringIndexer.getOutputCol())
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
print(label_stringIdx)
stages += [label_stringIdx]
numericCols = ['age','balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
print(assemblerInputs)
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

jobIndex
maritalIndex
educationIndex
defaultIndex
housingIndex
loanIndex
contactIndex
poutcomeIndex
StringIndexer_a9bceaafeb0f
['jobclassVec', 'maritalclassVec', 'educationclassVec', 'defaultclassVec', 'housingclassVec', 'loanclassVec', 'contactclassVec', 'poutcomeclassVec', 'age', 'balance', 'duration', 'campaign', 'pdays', 'previous']


Now use this pipe line using fit and transform passing df as data

In [16]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
df.show(12,True)

+---+-----------+--------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+--------+--------------+------------+---------------+--------------+-----------------+------------+---------------+------------+---------------+---------+-------------+------------+---------------+-------------+----------------+-----+--------------------+
|age|        job| marital|education|default|balance|housing|loan|contact|duration|campaign|pdays|previous|poutcome|deposit|jobIndex|   jobclassVec|maritalIndex|maritalclassVec|educationIndex|educationclassVec|defaultIndex|defaultclassVec|housingIndex|housingclassVec|loanIndex| loanclassVec|contactIndex|contactclassVec|poutcomeIndex|poutcomeclassVec|label|            features|
+---+-----------+--------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+--------+--------------+------------+---------------+--------------+-----------------+------------+---------------+----

In [18]:
print(df.select("features").show(5))
print(cols)

+--------------------+
|            features|
+--------------------+
|(30,[3,11,13,16,1...|
|(30,[3,11,13,16,1...|
|(30,[2,11,13,16,1...|
|(30,[4,11,13,16,1...|
|(30,[3,11,14,16,1...|
+--------------------+
only showing top 5 rows

None
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit']


In [19]:
# feature vector is in features  from assembler.
selectedCols = ['label', 'features'] + cols
dfSelected = df.select(selectedCols)
dfSelected.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [20]:
# this is code for multiple classification using logistic Regression
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LogisticRegression
train, test = dfSelected.randomSplit([0.7, 0.3], seed = 2018)
lr = LogisticRegression(maxIter=100, \

                        featuresCol="features", \

                        labelCol='label')
ovr = OneVsRest(classifier=lr, \
                labelCol='label', \
                featuresCol='features')

#from pyspark.ml import Pipeline
#pipeline_ovr = Pipeline(stages=[vecAssembler, stdScaler, ovr])
#pipelineModel_ovr = pipeline_ovr.fit(trainDF)

ovrModel = ovr.fit(train)
predictionsovr = ovrModel.transform(test)
predictionsovr.show(10)

+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+---------+--------+--------+-----+--------+--------+-------+--------------------+----------+
|label|            features|age|       job|marital|education|default|balance|housing|loan|  contact|duration|campaign|pdays|previous|poutcome|deposit|       rawPrediction|prediction|
+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+---------+--------+--------+-----+--------+--------+-------+--------------------+----------+
|  0.0|(30,[0,11,13,16,1...| 33|management|married|secondary|     no|    333|     no|  no| cellular|      80|       6|   -1|       0| unknown|     no|[1.91889531585926...|       0.0|
|  0.0|(30,[0,11,13,16,1...| 49|management|married|secondary|     no|   1601|     no|  no| cellular|      47|       5|   -1|       0| unknown|     no|[1.92010568650575...|       0.0|
|  0.0|(30,[0,11,13,16,1...| 52|management|married|secondary|     no|   5193|     no|

Decision Tree Classifier

In [21]:
from pyspark.ml.classification import DecisionTreeClassifier
train, test = dfSelected.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

#predictions.select("label","prediction").show(10)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)
print("Total Actual Positive",predictions.select("label").where('label == 1.0').count())
print("Total Actual Negative",predictions.select("label").where('label == 0.0').count())
pr = predictions.toPandas()
TruePositive =0
FalsePositive=0
TrueNegative=0
FalseNegative=0
Postive=1.0
Negative=0.0
pos=0
Neg=0

print("Total",len(pr["label"]))
for lbl in range(len(pr["label"])):
  if  pr["prediction"][lbl]==Postive:
    pos+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TruePositive+=1
    else:
      FalsePositive+=1
  if  pr["prediction"][lbl]==Negative:
    Neg+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TrueNegative+=1
    else:
      FalseNegative+=1
print("Total Positive & Negative in Prediction. Pos: ",pos,",Neg",Neg)
print("TruePostive",TruePositive,"FalsePostive",FalsePositive)
print("TrueNegative",TrueNegative,"FalseNegative",FalseNegative)

Training Dataset Count: 7855
Test Dataset Count: 3307
+---+----------+-----+--------------+----------+--------------------+
|age|       job|label| rawPrediction|prediction|         probability|
+---+----------+-----+--------------+----------+--------------------+
| 33|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 49|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 52|management|  0.0|[520.0,1931.0]|       1.0|[0.21215830273357...|
| 53|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 58|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 32|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 57|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 52|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 46|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
| 31|management|  0.0|[2498.0,481.0]|       0.0|[0.83853642161799...|
+---+----------+-----+--------------

AUC - ROC curve

is a performance measurement for the classification problems at various threshold settings. ROC is a probability curve and AUC represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes. Higher the AUC, the better the model is at predicting 0s as 0s and 1s as 1s. By analogy, the Higher the AUC, the better the model is at distinguishing between patients with the disease and no disease. An excellent model has AUC near to the 1 which means it has a good measure of separability. A poor model has AUC near to the 0 which means it has the worst measure of separability. In fact, it means it is reciprocating the result. It is predicting 0s as 1s and 1s as 0s. And when AUC is 0.5, it means the model has no class separation capacity whatsoever.

In [22]:
#predictions.show(10)
#print(dtModel.toDebugString)
#Evaluate our Decision Tree model.

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.7808118726917547


Bagging using Random Forest

observe the number of trees

In [23]:
#RandomForest
from pyspark.ml.classification import RandomForestClassifier
#rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rf = RandomForestClassifier(featuresCol='features', labelCol='label', maxDepth = 4,numTrees=20)
train, test = dfSelected.randomSplit([0.7, 0.3], seed = 2018)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
pr = predictions.toPandas()
print("Total Actual Positive",predictions.select("label").where('label == 1.0').count())
print("Total Actual Negative",predictions.select("label").where('label == 0.0').count())


Total Actual Positive 1603
Total Actual Negative 1704


In [24]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
TruePositive =0
FalsePositive=0
TrueNegative=0
FalseNegative=0
Postive=1.0
Negative=0.0
pos=0
Neg=0
print("Total",len(pr["label"]))
for lbl in range(len(pr["label"])):
  if  pr["prediction"][lbl]==Postive:
    pos+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TruePositive+=1
    else:
      FalsePositive+=1
  if  pr["prediction"][lbl]==Negative:
    Neg+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TrueNegative+=1
    else:
      FalseNegative+=1
print("Total positive & Negative in Predictions, Pos",pos,"Neg",Neg)
print("TruePostive",TruePositive,"FalsePostive",FalsePositive)
print("TrueNegative",TrueNegative,"FalseNegative",FalseNegative)

Test Area Under ROC: 0.8729169046301097
Total 3307
Total positive & Negative in Predictions, Pos 1497 Neg 1810
TruePostive 1214 FalsePostive 283
TrueNegative 1421 FalseNegative 389


In [25]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

ml = GBTClassifier(maxIter=10, featuresCol='features', labelCol='label',maxDepth = 10)
train, test = dfSelected.randomSplit([0.7, 0.2], seed = 2018)
mlModel = ml.fit(train)
predictions = mlModel.transform(test)

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))



Test Area Under ROC: 0.869695789478623


In [ ]:
# Calculate accuracy and F-1 score
#accuracy_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
#accuracy = accuracy_evaluator.evaluate(predictions.select('label', 'prediction'))

#f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
#f1_score = f1_score_evaluator.evaluate(predictions.select('label', 'prediction'))
#print(accuracy, f1_score)

In [27]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
gbt = GBTClassifier(maxIter=10)
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

0.8989346833243608

In [28]:
predictions.select('label', 'prediction').show(13)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 13 rows

